In [7]:
from surprise.model_selection import GridSearchCV
from surprise import SVD, Dataset, Reader

file_path = "/mnt/c/Users/Helin/OneDrive/Dokumente/BachelorThesis/code/srcCode/recsys-negative-feedback/data/raw/movielens/u.data"
reader = Reader(line_format="user item rating timestamp", sep="\t")

#which automates hyperparameter tuning using cross-validation

data = Dataset.load_from_file(file_path, reader=reader)

param_grid = {#defines hyperparameter values to test
    'n_factors': [10, 20, 50, 100, 150],
    'n_epochs': [10, 20, 30, 40],
    'lr_all': [0.001, 0.002, 0.005, 0.01],
    'reg_all': [0.005, 0.01, 0.02, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data) #try all combinations

print(gs.best_params['rmse'])

{'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}


Train SVD with each parameter combo.
Evaluate with RMSE and MAE.
Use 3-fold cross validation.

the result after 118min runtime:
{'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}







In [ ]:
from surprise import SVD
from surprise.model_selection import GridSearchCV
print("Surprise is working!")


Surprise is working!


In [9]:
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print()

# Test core libraries
import pandas as pd
import numpy as np
print(f"✓ pandas {pd.__version__}")
print(f"✓ numpy {np.__version__}")

# Test Surprise
from surprise import SVD
from surprise.model_selection import GridSearchCV
print(f"✓ surprise (scikit-surprise) is working!")

# Quick Surprise test
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')
algo = SVD(n_factors=50, n_epochs=20)
print(f"✓ Created SVD model: {algo}")

Python version: 3.8.20 (default, Oct  3 2024, 15:24:27) 
[GCC 11.2.0]
Python executable: /home/helin/miniconda3/envs/recsys-conda/bin/python

✓ pandas 1.4.2
✓ numpy 1.22.3
✓ surprise (scikit-surprise) is working!
✓ Created SVD model: <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7727f38395b0>


In [11]:
from surprise.model_selection import GridSearchCV
#which automates hyperparameter tuning using cross-validation
from surprise import SVD, Dataset, Reader


file_path = "/mnt/c/Users/Helin/OneDrive/Dokumente/BachelorThesis/code/srcCode/recsys-negative-feedback/data/raw/movielens/u.data"
reader = Reader(line_format="user item rating timestamp", sep="\t")

#which automates hyperparameter tuning using cross-validation

data = Dataset.load_from_file(file_path, reader=reader)

param_grid = {#defines hyperparameter values to test
     'n_factors': [...],
    'n_epochs': [...],
    'lr_all': [...],
    'reg_all': [...],
    'biased': [True, False],
    'init_std_dev': [0.01, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data) #try all combinations

print(gs.best_params['rmse'])

TypeError: must be real number, not ellipsis

Creates a GridSearchCV object:

SVD: the algorithm to tune.

param_grid: the hyperparameter combinations.

measures: evaluation metrics (Root Mean Squared Error and Mean Absolute Error).

cv=3: uses 3-fold cross-validation to ensure robustness.